In [1]:
let parseGrid lines =
    lines
    |> Array.map (
            Seq.map (
                fun c ->
                    match c with
                    | 'E' -> 25
                    | 'S' -> 0
                    | _ -> int c - int 'a'
            ) >> Array.ofSeq
        )

let positionOf ch lines =
    lines
    |> Array.indexed
    |> Array.pick (
        fun (r, l) ->
            l
            |> Seq.tryFindIndex ((=) ch)
            |> Option.map (fun c -> (r,c))
        )

let startPos lines = lines |> positionOf 'S'
let endPos lines = lines |> positionOf 'E'

let adjacentSquares (grid:_[][]) (r, c) =
    let h = grid.Length
    let w = grid[0].Length
    [
        if r > 0 then
            (r-1,c)
        if r < (h-1) then
            (r+1, c)
        if c > 0 then
            (r,c-1)
        if c < (w-1) then
            (r, c+1)
    ]

let canClimb (grid : _[][]) (fromR, fromC) (toR, toC) =
    grid[toR][toC] - grid[fromR][fromC] <= 1


In [2]:
type Position = int*int
type Node = { CostF : int; DistanceFromStartG : int; EstimateToEndH : int; Previous : Position option }

let astar getOutbound getEstimate endPos start =
    let rec astar (openNodes : Map<Position,Node>) (closedNodes : Map<Position,Node>) =
        let pos,node = openNodes |> Seq.minBy(fun kvp -> kvp.Value.CostF) |> (fun kvp -> kvp.Key, kvp.Value)

        let newOpen = openNodes |> Map.remove pos
        let newClosed = closedNodes |> Map.add pos node
        if pos = endPos then
            pos
            |> List.unfold (fun p -> newClosed.[p].Previous |> Option.map (fun p -> p,p))
            |> List.rev
        else
            let nextPositions =
                getOutbound pos
                |> Seq.filter (closedNodes.ContainsKey >> not)

            let newOpen =
                nextPositions
                |> Seq.fold (
                    fun newOpen childPos ->
                        let childDistanceFromStartG = 1 + node.DistanceFromStartG
                        let childEstimateToEndH     = getEstimate childPos endPos
                        let childCostF              = childDistanceFromStartG  + childEstimateToEndH

                        match Map.tryFind childPos newOpen with
                        | Some n when n.DistanceFromStartG < childDistanceFromStartG ->
                            newOpen
                        | _ ->
                            let childNode = { CostF = childCostF; DistanceFromStartG = childDistanceFromStartG; EstimateToEndH = childEstimateToEndH; Previous = Some pos }
                            newOpen |> Map.add childPos childNode
                ) newOpen

            astar newOpen newClosed

    let openNodes = (start, {CostF = 0; DistanceFromStartG = 0; EstimateToEndH = getEstimate start endPos; Previous = None}) |> Seq.singleton |> Map.ofSeq
    astar (openNodes : Map<Position,Node>) Map.empty


In [4]:
let solver grid (startR, startC) (endR, endC) =
    let getOutbound pos = adjacentSquares grid pos |> List.filter (canClimb grid pos)
    let estimate (r,c) (endR, endC) = abs (r - endR) + abs (c - endC)
    astar getOutbound estimate (endR, endC) (startR, startC)

In [5]:
#r "nuget: FsUnit"
open FsUnitTyped

let testInput = 
    [|
        "Sabqponm"
        "abcryxxl"
        "accszExk"
        "acctuvwj"
        "abdefghi"
    |]

let testGrid = parseGrid testInput
let testStart = startPos testInput
let testEnd = endPos testInput

testStart |> shouldEqual (0,0)
testEnd |> shouldEqual (2,5)

let shortest = solver testGrid testStart testEnd
shortest |> List.length |> shouldEqual 31


Installed Packages FsUnit, 5.1.0

In [7]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_12.txt")
let lines = 
    File.ReadAllLines(sourcePath)

let grid = parseGrid lines
let start = startPos lines
let ends = endPos lines

let shortest = solver grid start ends |> List.length


In [ ]:
printfn "Shortest path %d steps" shortest